# Import Widget Libraries

In [1]:
%load_ext autoreload   
%autoreload 2
import kyurem
from kyurem.core.idom_loader import reload_bundle
from kyurem.core.widget import WidgetModel

kyurem_client: 0.0.0


# Import Graph Querying Library

In [2]:
from kyurem import Service
service = Service(kh = 'indeed.v0.0.1:612e3403')

In [3]:
schema = service.get_kh_edge_list()

# Create an Overview/Exploration Widget

In [97]:
from kyurem import Explorer


def init(state): 
    return {
        "subgraph": None,
        "children": service.get_children_node_distributions(),
        "relations": service.get_relation_distribution(),
    }

def focus(state, node, panel):
    result = service.get_node_neighborhood(node)
    
    data = { "subgraph": None }
    if panel != "schema":
        data["schema"] = result["schema"]
    if panel != "children":
        data["children"] = service.get_children_node_distributions(
            node["node_label"], 
            node["node_property"], 
            node["node_property_value"]
        )

    data["relations"] = [
        {"x": relation["label"], "y": relation["count"], "type": type}
        for type, relations in result["relation_dist"].items()
        for relation in relations
    ]
    
    return data
  
# Create widget 
reload_bundle()
explorer = Explorer({
        "init": init,
        "focus": focus
    }, schema)

explorer.show()

component(7f440c6c8280, self=<kyurem.widgets.ReducerWidget.ReducerWidget object at 0x7f440c5d36a0>)

LayoutWidget(Layout(component(7f440c6c8280, self=<kyurem.widgets.ReducerWidget.ReducerWidget object at 0x7f440…

In [65]:
from kyurem.utils.mdump import mdump
print(mdump(explorer.history, 3))

[ {
    "action": {
      "name": "init",
      "args": (),
      "kwargs": {},
    },
    "state": {
      "did_init": True,
      "data": { ...4 items... },
      "is_loading": False,
    },
  } ]


In [ ]:
def filter_by_relation(state, type, direction):
    if state.nodelabel and state.nodetitle:
        node = {
            "node_label": state.nodelabel,
            "node_property": "title",
            "node_property_value": state.nodetitle,
        }
    else:
        node = None

    result = service.get_relation_neighborhood(node, {"type": type, "direction": direction})
    data = {}
    data['schema'] = result["schema"]
    if not node:
        data['children'] = [
            {"x": x, "y": y} for x, y in result["node_dist"].items()
        ]
    return data

# View 0: Send nodes rather than labels in schema

In [ ]:
schema

# View 0: Click a node on minimap

In [ ]:
node ={'node_label': 'occupation',
       'node_property': 'title', 
       'node_property_value': 'occupation'}

service.get_children_node_distributions('occupation', 'title', 'Healthcare Occupations')

# View 0: Click a bar on node_dist

In [ ]:
node ={'node_label': 'occupation',
       'node_property': 'title', 
       'node_property_value': 'Allied Health Occupations'}

In [ ]:
nn = service.get_node_neighborhood(node)
nn

# View 0: Click a bar on relation dist

In [ ]:
node ={'node_label': 'occupation',
       'node_property': 'title', 
       'node_property_value': 'Fire Alarm Technicians'}

relation = {}
relation['type'] = 'has_core_skill'
relation['direction'] = 'out'

In [ ]:
service.get_relation_neighborhood(node, relation)

In [ ]:
service.get_relation_neighborhood(None, relation)

# View 1: Click a node on zoomed in graph

# View 1: Click a node on node_dist

# View 1: Click a node on relation dist